In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as m
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report

In [2]:
df = pd.read_csv('df_events.csv')
df

/tmp/ipykernel_2530/42670684.py:1: DtypeWarning: Columns (9,11,12,13,14,15,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_events.csv')


,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,Адрес_x,UNOM,Дата и время завершения события во,Адрес_y,Год постройки,...,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,name,target
0,Открыт колодец,MVK,2022-08-30 14:41:53.000000,2022-08-30 15:40:26.565000,ВАО,внутригородская территория муниципальный округ...,2115191,незавершено,"Дом по адресу Семеновская М. ул., д.13",1928.0,...,4793.0,0.0,4793.0,кирпичный,0.0,0.0,Нежилой,0.0,открыт колодец,28
1,Отклонение ГВС ниже нормы ночью (мониторинг),ASUPR,2022-02-20 06:24:07.000000,2022-04-04 02:43:22.101000,ВАО,внутригородская территория муниципальный округ...,2404461,незавершено,"Дом по адресу Дмитриевского ул., д.5",2007.0,...,8412.0,0.0,8412.0,панельные,0.0,0.0,Нежилой,0.0,отклонение гвс ниже нормы ночью мониторинг,2
2,Отклонение ГВС ниже нормы днем (мониторинг),ASUPR,2022-03-07 21:04:01.000000,2022-03-09 09:33:24.894000,ВАО,внутригородская территория муниципальный округ...,2404461,незавершено,"Дом по адресу Дмитриевского ул., д.5",2007.0,...,8412.0,0.0,8412.0,панельные,0.0,0.0,Нежилой,0.0,отклонение гвс ниже нормы днем мониторинг,2
3,Температура ГВС ниже нормы,ASUPR,2022-03-05 22:13:33.000000,2022-03-05 19:14:00.190000,ВАО,внутригородская территория муниципальный округ...,2404461,незавершено,"Дом по адресу Дмитриевского ул., д.5",2007.0,...,8412.0,0.0,8412.0,панельные,0.0,0.0,Нежилой,0.0,температура гвс ниже нормы,2
4,Температура ГВС ниже нормы,ASUPR,2022-02-18 21:43:54.000000,2022-02-18 18:44:14.682000,ВАО,внутригородская территория муниципальный округ...,2404461,незавершено,"Дом по адресу Дмитриевского ул., д.5",2007.0,...,8412.0,0.0,8412.0,панельные,0.0,0.0,Нежилой,0.0,температура гвс ниже нормы,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052965,Засор канализации,EDC,2022-10-28 08:26:59.000000,2022-10-28 08:45:46.634000,ВАО,внутригородская территория муниципальный округ...,14257,2022-10-28 08:45:46.634000,"Дом по адресу Металлургов ул., д.44, к.2",1966.0,...,2583.0,2583.0,0.0,панельные,0.0,0.0,в эксплуатации,0.0,засор канализации,1
1052966,Недогрев ГВС,EDC,2022-11-02 17:51:03.000000,2022-11-02 19:42:59.116000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-02 19:42:59.116000,"Дом по адресу Металлургов ул., д.44, к.2",1966.0,...,2583.0,2583.0,0.0,панельные,0.0,0.0,в эксплуатации,0.0,недогрев гвс,2
1052967,Недогрев ГВС,EDC,2022-11-03 10:19:13.000000,2022-11-03 10:47:10.487000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-03 10:47:10.487000,"Дом по адресу Металлургов ул., д.44, к.2",1966.0,...,2583.0,2583.0,0.0,панельные,0.0,0.0,в эксплуатации,0.0,недогрев гвс,2
1052968,Недогрев ГВС,EDC,2022-11-03 10:20:33.000000,2022-11-03 10:47:15.575000,ВАО,внутригородская территория муниципальный округ...,14257,2022-11-03 10:47:15.575000,"Дом по адресу Металлургов ул., д.44, к.2",1966.0,...,2583.0,2583.0,0.0,панельные,0.0,0.0,в эксплуатации,0.0,недогрев гвс,2


In [3]:
df.UNOM.value_counts().index[:15]

Int64Index([  24036,   18821,   27932,   18703,    8600,   17991, 2402235,
              23815,   24595,   19301,   64134,   24191,   15756,   13641,
              18687],
           dtype='int64')

In [4]:
#осталвяем самые популярных unom
df = df[df.UNOM.isin(df.UNOM.value_counts().index[:15])]
df

,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,Адрес_x,UNOM,Дата и время завершения события во,Адрес_y,Год постройки,...,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,name,target
75299,Нет питания УСПД,ASUPR,2022-02-25 21:26:26.000000,2022-02-25 18:29:06.926000,ВАО,внутригородская территория муниципальный округ...,23815,незавершено,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,...,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,нет питания успд,1
75300,Нет питания УСПД,ASUPR,2022-03-30 02:05:48.000000,2022-03-30 09:24:10.562000,ВАО,внутригородская территория муниципальный округ...,23815,незавершено,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,...,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,нет питания успд,1
75301,Открыт шкаф УСПД,ASUPR,2022-02-25 15:22:55.000000,2022-02-25 12:25:58.356000,ВАО,внутригородская территория муниципальный округ...,23815,незавершено,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,...,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,открыт шкаф успд,1
75302,Нет питания УСПД,ASUPR,2022-02-27 07:10:24.000000,2022-02-27 04:19:40.045000,ВАО,внутригородская территория муниципальный округ...,23815,незавершено,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,...,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,нет питания успд,1
75303,Нет питания УСПД,ASUPR,2022-02-22 18:10:30.000000,2022-02-22 15:29:52.430000,ВАО,внутригородская территория муниципальный округ...,23815,незавершено,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,...,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,нет питания успд,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840952,Отсутствие освещения в местах общего пользования,EDC,2022-11-04 11:50:49.000000,2022-11-04 21:04:22.074000,ВАО,внутригородская территория муниципальный округ...,19301,2022-11-04 21:04:22.074000,"Дом по адресу Первомайская Ср. ул., д.13",1989.0,...,11265.0,11214.0,51.0,панельные,3.0,3.0,в эксплуатации,0.0,отсутствие освещения в местах общего пользования,4
840953,Повреждение элементов входной двери,NG,2022-11-06 16:07:27.792000,2022-11-07 12:41:55.089000,ВАО,внутригородская территория муниципальный округ...,19301,2022-11-07 12:41:55.089000,"Дом по адресу Первомайская Ср. ул., д.13",1989.0,...,11265.0,11214.0,51.0,панельные,3.0,3.0,в эксплуатации,0.0,повреждение элементов входной двери,1
840954,Повреждение элементов входной двери,EDC,2022-11-06 16:07:55.000000,2022-11-06 21:40:07.894000,ВАО,внутригородская территория муниципальный округ...,19301,2022-11-06 21:40:07.894000,"Дом по адресу Первомайская Ср. ул., д.13",1989.0,...,11265.0,11214.0,51.0,панельные,3.0,3.0,в эксплуатации,0.0,повреждение элементов входной двери,1
840955,Повреждение элементов входной двери,NG,2022-11-07 09:15:23.874000,2022-11-07 14:12:57.968000,ВАО,внутригородская территория муниципальный округ...,19301,2022-11-07 14:12:57.968000,"Дом по адресу Первомайская Ср. ул., д.13",1989.0,...,11265.0,11214.0,51.0,панельные,3.0,3.0,в эксплуатации,0.0,повреждение элементов входной двери,1


In [5]:
df = df.drop(['Наименование', 'Дата создания во внешней системе', 'Дата закрытия', 'Дата и время завершения события во', 'Округ', 'Адрес_x', 'UNOM', 'name'], axis=1)
df.head(50)

,Источник,Адрес_y,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,target
75299,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75300,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75301,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75302,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75303,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75304,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75305,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75306,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75307,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1
75308,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969.0,П-29,9.0,4.0,6718.0,5639.0,1079.0,кирпичные,4.0,0.0,в эксплуатации,0.0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39942 entries, 75299 to 840956
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Источник                             39942 non-null  object
 1   Адрес_y                              39942 non-null  object
 2   Год постройки                        39942 non-null  object
 3   Серия проекта                        39942 non-null  object
 4   Количество этажей                    39942 non-null  object
 5   Количество подъездов                 39942 non-null  object
 6   Общая площадь                        39942 non-null  object
 7   Общая площадь жилых помещений        39942 non-null  object
 8   Общая площадь нежилых помещений      39942 non-null  object
 9   Материал стен                        39942 non-null  object
 10  Количество пассажирских лифтов       39942 non-null  object
 11  Количество грузопассажирских лифтов 

In [7]:
#замена данных на инт
for col in ['Год постройки', 'Количество этажей', 'Количество подъездов', 'Количество пассажирских лифтов',
         'Количество грузопассажирских лифтов', 'Количество грузовых лифтов']:
    df.loc[df[col] == 'нет данных', col] = '0.0'
    df[col] = df[col].astype('float32').astype('int32')

In [8]:
#замена данных на флоат
for col in ['Общая площадь', 'Общая площадь жилых помещений', 'Общая площадь нежилых помещений']:
    df.loc[df[col] == 'нет данных', col] = '0.0'
    df[col] = df[col].astype('float32').astype('int32')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39942 entries, 75299 to 840956
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Источник                             39942 non-null  object
 1   Адрес_y                              39942 non-null  object
 2   Год постройки                        39942 non-null  int32 
 3   Серия проекта                        39942 non-null  object
 4   Количество этажей                    39942 non-null  int32 
 5   Количество подъездов                 39942 non-null  int32 
 6   Общая площадь                        39942 non-null  int32 
 7   Общая площадь жилых помещений        39942 non-null  int32 
 8   Общая площадь нежилых помещений      39942 non-null  int32 
 9   Материал стен                        39942 non-null  object
 10  Количество пассажирских лифтов       39942 non-null  int32 
 11  Количество грузопассажирских лифтов 

In [10]:
df.loc[df['Серия проекта'] == 'нет данных', 'Серия проекта'] = 'нет серии проекта'
df.loc[df['Материал стен'] == 'нет данных', 'Материал стен'] = 'нет информации по материалам стен'

In [11]:
df = df.loc[df['Статус МКД'] != 'снесен']

In [12]:
df[['Источник', 'Серия проекта']]

,Источник,Серия проекта
75299,ASUPR,П-29
75300,ASUPR,П-29
75301,ASUPR,П-29
75302,ASUPR,П-29
75303,ASUPR,П-29
...,...,...
840952,EDC,П-44
840953,NG,П-44
840954,EDC,П-44
840955,NG,П-44


In [13]:
df

,Источник,Адрес_y,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов,target
75299,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969,П-29,9,4,6718,5639,1079,кирпичные,4,0,в эксплуатации,0,1
75300,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969,П-29,9,4,6718,5639,1079,кирпичные,4,0,в эксплуатации,0,1
75301,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969,П-29,9,4,6718,5639,1079,кирпичные,4,0,в эксплуатации,0,1
75302,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969,П-29,9,4,6718,5639,1079,кирпичные,4,0,в эксплуатации,0,1
75303,ASUPR,"Дом по адресу Сиреневый бульв., д.73, к.1",1969,П-29,9,4,6718,5639,1079,кирпичные,4,0,в эксплуатации,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840952,EDC,"Дом по адресу Первомайская Ср. ул., д.13",1989,П-44,17,3,11265,11214,51,панельные,3,3,в эксплуатации,0,4
840953,NG,"Дом по адресу Первомайская Ср. ул., д.13",1989,П-44,17,3,11265,11214,51,панельные,3,3,в эксплуатации,0,1
840954,EDC,"Дом по адресу Первомайская Ср. ул., д.13",1989,П-44,17,3,11265,11214,51,панельные,3,3,в эксплуатации,0,1
840955,NG,"Дом по адресу Первомайская Ср. ул., д.13",1989,П-44,17,3,11265,11214,51,панельные,3,3,в эксплуатации,0,1


## Обучение модели

In [14]:
x_train, x_test, y_train, y_test = train_test_split(\
    df.drop(['target'], axis = 1), df['target'], test_size = 0.30, random_state = 13)
x_test

,Источник,Адрес_y,Год постройки,Серия проекта,Количество этажей,Количество подъездов,Общая площадь,Общая площадь жилых помещений,Общая площадь нежилых помещений,Материал стен,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Статус МКД,Количество грузовых лифтов
795583,ASUPR,"Дом по адресу Парковая 13-я ул., д.26",2002,П-44т,17,5,19212,19045,167,панельные,5,5,в эксплуатации,0
560673,ASUPR,"Дом по адресу Остроумовская Б. ул., д.21",1979,индивидуальный проект,14,1,5383,5334,49,кирпичные,1,1,в эксплуатации,0
770397,ASUPR,"Дом по адресу Магнитогорская ул., д.21",1977,П-47,12,4,8779,8763,16,панельные,8,0,в эксплуатации,0
501281,ASUPR,"Дом по адресу Соколиной Горы 9-я ул., д.21",1970,П-18/22,12,1,4854,3472,1382,из железобетонных сегментов,2,0,в эксплуатации,0
555302,ASUPR,"Дом по адресу Сталеваров ул., д.8/22, к.2",1972,П-49 Д,9,2,3855,3855,0,панельные,2,0,в эксплуатации,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132729,ASUPR,"Дом по адресу Снайперская ул., д.6, к.2",1970,II-49-04/М вар Д,9,4,7487,7487,0,панельные,4,0,в эксплуатации,0
128774,ASUPR,"Дом по адресу Снайперская ул., д.6, к.2",1970,II-49-04/М вар Д,9,4,7487,7487,0,панельные,4,0,в эксплуатации,0
222566,ASUPR,"Дом по адресу Парковая 11-я ул., д.52, к.5",1963,П-32,5,4,3576,3554,22,из железобетонных сегментов,0,0,в эксплуатации,0
490323,ASUPR,"Дом по адресу Измайловская пл., д.9, к.1",1954,нет серии проекта,5,4,7182,5456,1726,кирпичные,0,0,в эксплуатации,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39942 entries, 75299 to 840956
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Источник                             39942 non-null  object
 1   Адрес_y                              39942 non-null  object
 2   Год постройки                        39942 non-null  int32 
 3   Серия проекта                        39942 non-null  object
 4   Количество этажей                    39942 non-null  int32 
 5   Количество подъездов                 39942 non-null  int32 
 6   Общая площадь                        39942 non-null  int32 
 7   Общая площадь жилых помещений        39942 non-null  int32 
 8   Общая площадь нежилых помещений      39942 non-null  int32 
 9   Материал стен                        39942 non-null  object
 10  Количество пассажирских лифтов       39942 non-null  int32 
 11  Количество грузопассажирских лифтов 

In [16]:
text_features = ['Адрес_y']
cat_features = ['Источник', 'Серия проекта', 'Количество этажей', 'Количество подъездов',
               'Материал стен', 'Статус МКД']

In [17]:
train_pool = Pool(x_train, y_train, cat_features=cat_features, text_features=text_features)

In [18]:
%%time
model = CatBoostClassifier(loss_function='MultiClass', random_seed=13)
model.fit(train_pool, verbose = 100)
pred = model.predict(x_test)

Learning rate set to 0.093777
0:	learn: 1.6251673	total: 424ms	remaining: 7m 3s
100:	learn: 0.1350374	total: 1m 21s	remaining: 12m 2s
200:	learn: 0.1234491	total: 2m 51s	remaining: 11m 20s
300:	learn: 0.1139149	total: 4m 9s	remaining: 9m 39s
400:	learn: 0.1059784	total: 5m 37s	remaining: 8m 24s
500:	learn: 0.0992240	total: 6m 59s	remaining: 6m 57s
600:	learn: 0.0932433	total: 8m 22s	remaining: 5m 33s
700:	learn: 0.0884724	total: 9m 40s	remaining: 4m 7s
800:	learn: 0.0840088	total: 10m 57s	remaining: 2m 43s
900:	learn: 0.0800726	total: 12m 13s	remaining: 1m 20s
999:	learn: 0.0762831	total: 13m 28s	remaining: 0us
CPU times: user 49min 58s, sys: 6.22 s, total: 50min 4s
Wall time: 13min 28s


In [19]:
print('F1 score -', f1_score(y_test, pred, average=None))

F1 score - [0.         0.97787648 0.         0.         0.         0.66666667
 0.         0.         0.         0.         0.         0.09565217
 0.         0.        ]


In [20]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.96      1.00      0.98     11452
           2       0.00      0.00      0.00        59
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00       156
           5       0.50      1.00      0.67         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00        21
           9       0.00      0.00      0.00        10
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00        47
          13       0.65      0.05      0.10       213
          25       0.00      0.00      0.00        17
          28       0.00      0.00      0.00         2

    accuracy                           0.96     11983
   macro avg       0.15      0.15      0.12     11983
weighted avg       0.93      0.96      0.94     11983



/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/localadmin/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
